In [1]:
from pathlib import Path

from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer

/anaconda/envs/codetf/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/anaconda/envs/codetf/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//eastus.api.azureml.ms/mlflow/v1.0/subscriptions/60089760-5d77-4a24-b1f4-03f3eb78acf6/resourceGroups/azure-test-resource-group/providers/Microsoft.MachineLearningServices/workspaces/mdl'), PosixPath('azureml')}
  warn(msg)
/anaconda/envs/codetf/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//127.0.0.1'), PosixPath('http'), PosixPath('46808/OBO/token')}
  warn(msg)
/anaconda/envs/codetf/lib/python3.10/site-pack


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /anaconda/envs/codetf/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 3.7
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /anaconda/envs/codetf/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113_nocublaslt.so...


In [2]:
root_dir = Path.cwd().parent

In [3]:
datasets = load_dataset("json", data_files={
    "train": str(root_dir / "data" / "train.jsonl")
}, cache_dir=root_dir / "data" / "cache", split="train")

Found cached dataset json (/mnt/batch/tasks/shared/LS_root/mounts/clusters/mdl/code/Users/Paul.Brauckmann/mdl-ii/src/data/cache/json/default-62e25f22c21bb7d6/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


In [4]:
# datasets = load_dataset("code_x_glue_ct_code_to_text", 'python', split="train")
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5p-770m-py")

def preprocess_function(examples):
    source = [' '.join(ex) for ex in examples["code_tokens"]]
    target = [' '.join(ex) for ex in examples["docstring_tokens"]]

    model_inputs = tokenizer(source, max_length=320, padding="max_length", truncation=True)
    labels = tokenizer(target, max_length=128, padding="max_length", truncation=True)

    model_inputs["labels"] = labels["input_ids"].copy()
    model_inputs["labels"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in model_inputs["labels"]
    ]
    return model_inputs

train_data = datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=datasets.column_names,
    num_proc=64,
    load_from_cache_file=False,
)

In [5]:
model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5p-770m-py")
model.to("cuda")

training_args = TrainingArguments(
    output_dir=root_dir / "modeling" / "models" / "codet5p_770m_py",
    overwrite_output_dir=False,

    do_train=True,
    save_strategy='epoch',

    num_train_epochs=2,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,

    learning_rate=5e-5,
    weight_decay=0.05,
    warmup_steps=200,

    logging_dir=root_dir / "modeling" / "models" / "codet5p_770m_py",
    logging_first_step=True,
    logging_steps=100,
    save_total_limit=1,

    dataloader_drop_last=True,
    dataloader_num_workers=4,

    local_rank=-1,
    deepspeed=None,
    fp16=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model(root_dir / "modeling" / "models" / "codet5p_770m_py")